In [29]:
import numpy as np
import msgpack as mp
import msgpack_numpy as mpn
import cv2
from cv2 import aruco
import os
import sys
import toml
import tqdm
from ultralytics import YOLO

In [30]:
_pth = toml.load("DATA_PATH.toml")["data_path"]["directory"]
_pth = os.path.dirname(_pth)
_parent_folder = "paper"
_calib_folder_name = "calibration_00"

_base_pth = os.path.join(_pth, "recorded_data", _parent_folder)
# _folder_list = os.listdir(_base_pth)[10:15] # forward
_folder_list = os.listdir(_base_pth)[15:]  # sideways
# _folder_list = os.listdir(_base_pth)[25:] # spots
_folder_name = _folder_list[0]


_webcam_calib_folder = os.path.join(
    _pth, "recorded_data", _parent_folder, _calib_folder_name
)
_webcam_calib_folder = os.path.join(_webcam_calib_folder)
_webcam_calib_pth = os.path.join(_webcam_calib_folder, "webcam_calibration.msgpack")

with open(_webcam_calib_pth, "rb") as f:
    webcam_calib = mp.Unpacker(f, object_hook=mpn.decode)
    _temp = next(webcam_calib)
    _webcam_cam_mat = _temp[0]
    _webcam_dist = _temp[1]

ar_lframe_pth = os.path.join(_webcam_calib_folder, "spots_rotmat.msgpack")
with open(ar_lframe_pth, "rb") as f:
    ar_lframe = mp.Unpacker(f, object_hook=mpn.decode)
    _ar_lframe_rot = next(ar_lframe)
    _ar_lframe_org = next(ar_lframe)

## Estimate marker pose

In [31]:
def estimatePoseSingleMarkers(corners, marker_size, mtx, distortion):
    marker_points = np.array(
        [
            [-marker_size / 2, marker_size / 2, 0],
            [marker_size / 2, marker_size / 2, 0],
            [marker_size / 2, -marker_size / 2, 0],
            [-marker_size / 2, -marker_size / 2, 0],
        ],
        dtype=np.float32,
    )
    trash = []
    rvecs = []
    tvecs = []
    for c in corners:
        nada, R, t = cv2.solvePnP(
            marker_points, c, mtx, distortion, True, flags=cv2.SOLVEPNP_ITERATIVE
        )

        if not (R is None or t is None):
            R = np.array(R).reshape(1, 3).tolist()
            t = np.array(t).reshape(1, 3).tolist()

        rvecs.append(R)
        tvecs.append(t)
    return rvecs, tvecs

In [32]:
ARUCO_PARAMETERS = aruco.DetectorParameters()
ARUCO_DICT = aruco.getPredefinedDictionary(aruco.DICT_ARUCO_MIP_36H12)
detector = aruco.ArucoDetector(ARUCO_DICT, ARUCO_PARAMETERS)
markerLength = int(_folder_name.split("_")[2]) / 100
markerSeperation = 0.01

board = aruco.GridBoard(
    size=[1, 1],
    markerLength=markerLength,
    markerSeparation=markerSeperation,
    dictionary=ARUCO_DICT,
)

In [33]:
# Create vectors we'll be using for rotations and translations for postures
rotation_vectors, translation_vectors = None, None
axis = np.float32(
    [
        [-0.5, -0.5, 0],
        [-0.5, 0.5, 0],
        [0.5, 0.5, 0],
        [0.5, -0.5, 0],
        [-0.5, -0.5, 1],
        [-0.5, 0.5, 1],
        [0.5, 0.5, 1],
        [0.5, -0.5, 1],
    ]
)

In [34]:
_video_pth = os.path.join(_base_pth, _folder_name, "webcam_color.msgpack")
_video_file = open(_video_pth, "rb")
_video_data = mp.Unpacker(_video_file, object_hook=mpn.decode)

# open video file for writing
_video_file_name = _folder_name + "_ar.mp4"
_video_file_name = os.path.join(_base_pth, _folder_name, _video_file_name)
_new_video = cv2.VideoWriter(
    _video_file_name, cv2.VideoWriter_fourcc(*"mp4v"), 3, (1280, 720)
)

for _frame in _video_data:
    shape = _frame.shape
    gray = cv2.cvtColor(_frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedpoints = detector.detectMarkers(_frame)
    # refine corners
    corners, ids, rejectedpoints, _ = detector.refineDetectedMarkers(
        image=_frame,
        board=board,
        detectedCorners=corners,
        detectedIds=ids,
        rejectedCorners=rejectedpoints,
        cameraMatrix=_webcam_cam_mat,
        distCoeffs=_webcam_dist,
    )
    rotation_vectors, translation_vectors = estimatePoseSingleMarkers(
        corners=corners,
        marker_size=markerLength,
        mtx=_webcam_cam_mat,
        distortion=_webcam_dist,
    )

    for rvec, tvec in zip(rotation_vectors, translation_vectors):
        rvec = np.array(rvec)
        tvec = np.array(tvec)
        _frame = aruco.drawDetectedMarkers(_frame, corners=corners, ids=ids)

        _frame = cv2.drawFrameAxes(
            _frame, _webcam_cam_mat, _webcam_dist, rvec, tvec, 0.05
        )

    _new_video.write(_frame)

    cv2.imshow("frame", _frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

_new_video.release()
_video_file.close()
cv2.destroyAllWindows()

## YOLO

In [35]:
model = YOLO(r"D:\CMC\DeepVision\models_save\mip_p2_60e_noise.pt")

In [36]:
_video_pth = os.path.join(_base_pth, _folder_name, "webcam_color.msgpack")
_video_file = open(_video_pth, "rb")
_video_data = mp.Unpacker(_video_file, object_hook=mpn.decode)

# open video file for writing
_video_file_name = _folder_name + "_yolo.mp4"
_video_file_name = os.path.join(_base_pth, _folder_name, _video_file_name)
_new_video = cv2.VideoWriter(
    _video_file_name, cv2.VideoWriter_fourcc(*"mp4v"), 3, (1280, 720)
)

for _frame in _video_data:
    shape = _frame.shape
    yolo_results = model.predict(_frame, verbose=False)[0]
    _frame = yolo_results.plot()

    _new_video.write(_frame)

    cv2.imshow("frame", _frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

_new_video.release()
_video_file.close()
cv2.destroyAllWindows()